##### Copyright 2020 The Cirq Developers

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Noise

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.example.org/cirq/noise"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on QuantumLib</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/noise.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/noise.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/noise.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [2]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

In [3]:
import cirq

For simulation, it is useful to have `Gate` objects that enact noisy quantum evolution. Cirq supports modeling noise via *operator sum* representations of noise (these evolutions are also known as quantum operations or quantum dynamical maps). 

This formalism models evolution of the density matrix $\rho$ via

$$
\rho \rightarrow \sum_{k = 1}^{m} A_k \rho A_k^\dagger
$$

where $A_k$ are known as *Kraus operators*. These operators are not necessarily unitary but must satisfy the trace-preserving property

$$
\sum_k A_k^\dagger A_k = I .
$$

A channel with $m = 1$ unitary Kraus operator is called *coherent* (and is equivalent to a unitary gate operation), otherwise the channel is called *incoherent*. For a given noisy channel, Kraus operators are not necessarily unique. For more details on these operators, see [John Preskill's lecture notes](http://theory.caltech.edu/~preskill/ph219/chap3_15.pdf).

## Common channels

Cirq defines many commonly used quantum channels in [`ops/common_channels.py`](https://github.com/quantumlib/Cirq/blob/master/cirq/ops/common_channels.py). For example, the single-qubit bit-flip channel

$$
\rho \rightarrow (1 - p) \rho + p X \rho X
$$

with parameter $p = 0.1$ can be created as follows.

In [4]:
"""Get a single-qubit bit-flip channel."""
bit_flip = cirq.bit_flip(p=0.1)

To see the Kraus operators of a channel, the `cirq.channel` protocol can be used.

In [5]:
for i, kraus in enumerate(cirq.channel(bit_flip)):
    print(f"Kraus operator {i + 1} is:\n", kraus, end="\n\n")

Kraus operator 1 is:
 [[0.9486833 0.       ]
 [0.        0.9486833]]

Kraus operator 2 is:
 [[0.        +0.j 0.31622777+0.j]
 [0.31622777+0.j 0.        +0.j]]



As mentioned, all channels are subclasses of `cirq.Gate`s. As such, they can act on qubits and be used in circuits in the same manner as gates.

In [6]:
"""Example of using channels in a circuit."""
# See the number of qubits a channel acts on.
nqubits = bit_flip.num_qubits()
print(f"Bit flip channel acts on {nqubits} qubit(s).\n")

# Apply the channel to each qubit in a circuit.
circuit = cirq.Circuit(
    bit_flip.on_each(cirq.LineQubit.range(3))
)
print(circuit)

Bit flip channel acts on 1 qubit(s).

0: ───BF(0.1)───

1: ───BF(0.1)───

2: ───BF(0.1)───


Channels can even be controlled.

In [7]:
"""Example of controlling a channel."""
# Get the controlled channel.
controlled_bit_flip = bit_flip.controlled(num_controls=1)

# Use it in a circuit.
circuit = cirq.Circuit(
    controlled_bit_flip(*cirq.LineQubit.range(2))
)
print(circuit)

0: ───@─────────
      │
1: ───BF(0.1)───


In addition to the bit-flip channel, other common channels predefined in Cirq are shown below. Definitions of these channels can be found in their docstrings - e.g., `help(cirq.depolarize)`.

* `cirq.phase_flip`
* `cirq.phase_damp`
* `cirq.amplitude_damp`
* `cirq.depolarize`
* `cirq.asymmetric_depolarize`
* `cirq.reset`

For example, the asymmetric depolarizing channel is defined by

$$
\rho \rightarrow (1-p_x-p_y-p_z) \rho + p_x X \rho X + p_y Y \rho Y + p_z Z \rho Z
$$

and can be instantiated as follows.

In [8]:
"""Get an asymmetric depolarizing channel."""
depo = cirq.asymmetric_depolarize(
    p_x=0.10,
    p_y=0.05,
    p_z=0.15,
)

circuit = cirq.Circuit(
    depo.on_each(cirq.LineQubit(0))
)
print(circuit)

0: ───A(0.1,0.05,0.15)───


## The `channel` and `mixture` protocols

We have seen the `cirq.channel` protocol which returns the Kraus operators of a channel. Some channels have the interpretation of randomly applying a single unitary Kraus operator $U_k$ with probability $p_k$, namely

$$
\rho \rightarrow \sum_k p_k U_k \rho U_k^\dagger {\rm ~where~} \sum_k p_k =1 {\rm ~and~ U_k U_k^\dagger= I}.
$$

For example, the bit-flip channel from above

$$
\rho \rightarrow (1 - p) \rho + p X \rho X
$$

can be interpreted as doing nothing (applying identity) with probability $1 - p$ and flipping the bit (applying $X$) with probability $p$. Channels with these interpretations support the `cirq.mixture` protocol. This protocol returns the probabilities and unitary Kraus operators of the channel.

In [9]:
"""Example of using the mixture protocol."""
for prob, kraus in cirq.mixture(bit_flip):
    print(f"With probability {prob}, apply\n", kraus, end="\n\n")

With probability 0.9, apply
 [[1. 0.]
 [0. 1.]]

With probability 0.1, apply
 [[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]



Channels that do not have this interpretation do not support the `cirq.mixture` protocol. Such channels apply Kraus operators with probabilities that depend on the state $\rho$. 

An example of a channel which does not support the mixture protocol is the amplitude damping channel with parameter $\gamma$ defined by Kraus operators

$$
M_0 = \begin{bmatrix} 1 & 0  \cr 0 & \sqrt{1 - \gamma} \end{bmatrix} 
\text{and }
M_1 = \begin{bmatrix} 0 & \sqrt{\gamma} \cr 0 & 0 \end{bmatrix} .
$$

In [10]:
"""The amplitude damping channel is an example of a channel without a mixture."""
channel = cirq.amplitude_damp(0.1)

if cirq.has_mixture(channel):
    print(f"Channel {channel} has a _mixture_ or _unitary_ method.")
else:
    print(f"Channel {channel} does not have a _mixture_ or _unitary_ method.")

Channel amplitude_damp(gamma=0.1) does not have a _mixture_ or _unitary_ method.


To summarize:

* Every channel supports the `cirq.channel` protocol. (Note this includes gates like `cirq.X` and `cirq.CNOT`.)
  - If magic method `_channel_` is not defined, `cirq.channel` looks for `_mixture_` then for `_unitary_`.
* A subset of channels which support `cirq.channel` also support the `cirq.mixture` protocol.
  - If magic method `_mixture_` is not defined, `cirq.mixture` looks for `_unitary_`.
* A subset of channels which support `cirq.mixture` also support the `cirq.unitary` protocol.
  - If `_unitary_` is not defined for an object, the object is probably not meant for a circuit!

## Custom channels

Channels not defined in `cirq.ops.common_channels` can be user-defined. Defining custom channels is similar to defining custom gates.

A minimal example for defining the channel

$$
\rho \mapsto (1 - p) \rho + p Y \rho Y
$$

is shown below.

In [11]:
"""Minimal example of defining a custom channel."""
class BitAndPhaseFlipChannel(cirq.SingleQubitGate):
    def __init__(self, p: float) -> None:
        self._p = p
    
    def _mixture_(self):
        ps = [1.0 - self._p, self._p]
        ops = [cirq.unitary(cirq.I), cirq.unitary(cirq.Y)]
        return tuple(zip(ps, ops))
    
    def _has_mixture_(self) -> bool:
        return True
    
    def _circuit_diagram_info_(self, args) -> str:
        return f"BitAndPhaseFlip({self._p})"

Note: The `_has_mixture_` magic method is not strictly required but is recommended.

We can now instantiate this channel and get its mixture:

In [12]:
"""Custom channels can be used like any other channels."""
bit_phase_flip = BitAndPhaseFlipChannel(p=0.05)

for prob, kraus in cirq.mixture(bit_phase_flip):
    print(f"With probability {prob}, apply\n", kraus, end="\n\n")

With probability 0.95, apply
 [[1. 0.]
 [0. 1.]]

With probability 0.05, apply
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]



Note: Since `_mixture_` is defined, the `cirq.channel` protocol can also be used.

The custom channel can be used in a circuit just like other predefined channels.

In [13]:
"""Example of using a custom channel in a circuit."""
circuit = cirq.Circuit(
    bit_phase_flip.on_each(*cirq.LineQubit.range(3))
)
circuit

0: ───BitAndPhaseFlip(0.05)───

1: ───BitAndPhaseFlip(0.05)───

2: ───BitAndPhaseFlip(0.05)───

This method of defining custom channels is the most general, but simple channels such as this can also be created directly from a `Gate` with the convenient `Gate.with_probability` method.

In [14]:
"""Create a channel with Gate.with_probability."""
channel = cirq.Y.with_probability(probability=0.05)

This produces the same mixture as the custom `BitAndPhaseFlip` channel above.

In [15]:
for prob, kraus in cirq.mixture(channel):
    print(f"With probability {prob}, apply\n", kraus, end="\n\n")

With probability 0.05, apply
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]

With probability 0.95, apply
 [[1. 0.]
 [0. 1.]]



Note that the order of Kraus operators is reversed from above, but this of course does not affect the action of the channel.

## Simulating noisy circuits

The `cirq.DensityMatrixSimulator` can simulate any noisy circuit (i.e., can apply any quantum channel) because it stores the full density matrix $\rho$. This simulation strategy updates the state $\rho$ by directly applying the Kraus operators of each quantum channel.

In [16]:
"""Simulating a circuit with the density matrix simulator."""
# Get a circuit.
qbit = cirq.GridQubit(0, 0)
circuit = cirq.Circuit(
    cirq.X(qbit),
    cirq.amplitude_damp(0.1).on(qbit)
)

# Display it.
print("Simulating circuit:")
print(circuit)

# Simulate with the density matrix simulator.
dsim = cirq.DensityMatrixSimulator()
rho = dsim.simulate(circuit).final_density_matrix

# Display the final density matrix.
print("\nFinal density matrix:")
print(rho)

Simulating circuit:
(0, 0): ───X───AD(0.1)───

Final density matrix:
[[0.1       +0.j 0.        +0.j]
 [0.        +0.j 0.90000004+0.j]]


Note that the density matrix simulator supports the `run` method which only gives access to measurements as well as the `simulate` method (used above) which gives access to the full density matrix.

Noisy circuits with arbitrary channels can also be simulated with the `cirq.Simulator`. When simulating such a channel, a single Kraus operator is randomly sampled (according to the probability distribution) and applied to the wavefunction. This method is known as "Monte Carlo (wavefunction) simulation" or "quantum trajectories."

Note: For channels which do not support the `cirq.mixture` protocol, the probability of applying each Kraus operator depends on the state. In contrast, for channels which do support the `cirq.mixture` protocol, the probability of applying each Kraus operator is independent of the state.

In [17]:
"""Simulating a noisy circuit via Monte Carlo simulation."""
# Get a circuit.
qbit = cirq.NamedQubit("Q")
circuit = cirq.Circuit(cirq.bit_flip(p=0.5).on(qbit))

# Display it.
print("Simulating circuit:")
print(circuit)

# Simulate with the density matrix simulator.
sim = cirq.Simulator()
psi = sim.simulate(circuit).dirac_notation()

# Display the final wavefunction.
print("\nFinal wavefunction:")
print(psi)

Simulating circuit:
Q: ───BF(0.5)───

Final wavefunction:
|0⟩


To see that the output is stochastic, you can run the cell above multiple times. Since $p = 0.5$ in the bit-flip channel, you should get $|0\rangle$ roughly half the time and $|1\rangle$ roughly half the time. The `run` method with many repetitions can also be used to see this behavior.

In [18]:
"""Example of Monte Carlo wavefunction simulation with the `run` method."""
circuit = cirq.Circuit(
    cirq.bit_flip(p=0.5).on(qbit),
    cirq.measure(qbit),
)
res = sim.run(circuit, repetitions=100)
print(res.histogram(key=qbit))

Counter({0: 51, 1: 49})


### Adding noise to circuits

Often circuits are defined with just unitary operations, but we want to simulate them with noise. There are several methods for inserting noise in Cirq.

For any circuit, the `with_noise` method can be called to insert a channel after every moment.

In [19]:
"""One method to insert noise in a circuit."""
# Define some noiseless circuit.
circuit = cirq.testing.random_circuit(
    qubits=3, n_moments=3, op_density=1, random_state=11
)

# Display the noiseless circuit.
print("Circuit without noise:")
print(circuit)

# Add noise to the circuit.
noisy = circuit.with_noise(cirq.depolarize(p=0.01))

# Display it.
print("\nCircuit with noise:")
print(noisy)

Circuit without noise:
              ┌──┐
0: ───@───X─────×────
      │   │     │
1: ───@───┼────S┼────
          │     │
2: ───Z───@─────×────
              └──┘

Circuit with noise:
                                                                        ┌──┐
0: ───@───D(0.01)[cirq.VirtualTag()]───X───D(0.01)[cirq.VirtualTag()]─────×────D(0.01)[cirq.VirtualTag()]───
      │                                │                                  │
1: ───@───D(0.01)[cirq.VirtualTag()]───┼───D(0.01)[cirq.VirtualTag()]────S┼────D(0.01)[cirq.VirtualTag()]───
                                       │                                  │
2: ───Z───D(0.01)[cirq.VirtualTag()]───@───D(0.01)[cirq.VirtualTag()]─────×────D(0.01)[cirq.VirtualTag()]───
                                                                        └──┘


This circuit can then be simulated using the methods described above.

Another technique is to pass a noise channel to the density matrix simulator as shown below.

In [20]:
"""Define a density matrix simulator with a noise model."""
noisy_dsim = cirq.DensityMatrixSimulator(
    noise=cirq.generalized_amplitude_damp(p=0.1, gamma=0.5)
)

Other than these general methods, channels can be added to circuits at any moment just as gates are. The channels can be different, be correlated, act on a subset of qubits, be custom defined, etc.

In [21]:
"""Defining a circuit with multiple noisy channels."""
qreg = cirq.LineQubit.range(4)
circ = cirq.Circuit(
    cirq.H.on_each(qreg),
    cirq.depolarize(p=0.01).on_each(qreg),
    cirq.qft(*qreg),
    bit_phase_flip.on_each(qreg[1::2]),
    cirq.qft(*qreg, inverse=True),
    cirq.reset(qreg[1]),
    cirq.measure(*qreg),
    cirq.bit_flip(p=0.07).controlled(1).on(*qreg[2:]),
)

print("Circuit with multiple channels:\n")
print(circ)

Circuit with multiple channels:

0: ───H───D(0.01)───qft───────────────────────────qft^-1───────M──────────────
                    │                             │            │
1: ───H───D(0.01)───#2────BitAndPhaseFlip(0.05)───#2───────R───M──────────────
                    │                             │            │
2: ───H───D(0.01)───#3────────────────────────────#3───────────M───@──────────
                    │                             │            │   │
3: ───H───D(0.01)───#4────BitAndPhaseFlip(0.05)───#4───────────M───BF(0.07)───


Circuits can also be modified with standard methods like `insert` to add channels at any point in the circuit. For example, to model simple state preparation errors, one can add bit-flip channels to the start of the circuit as follows.

In [22]:
"""Example of inserting channels in circuits."""
circ.insert(0, cirq.bit_flip(p=0.1).on_each(qreg))
print(circ)

0: ───BF(0.1)───H───D(0.01)───qft───────────────────────────qft^-1───────M──────────────
                              │                             │            │
1: ───BF(0.1)───H───D(0.01)───#2────BitAndPhaseFlip(0.05)───#2───────R───M──────────────
                              │                             │            │
2: ───BF(0.1)───H───D(0.01)───#3────────────────────────────#3───────────M───@──────────
                              │                             │            │   │
3: ───BF(0.1)───H───D(0.01)───#4────BitAndPhaseFlip(0.05)───#4───────────M───BF(0.07)───
